In [138]:
import os
import cv2
from roboflow import Roboflow
import re
import yaml
import shutil


In [139]:
RF_KEY  = os.environ.get("RF_KEY", "")
RF_WORKSPACE = os.environ.get("RF_WORKSPACE", "")
RF_PROJECT = os.environ.get("RF_LABELLER_PROJECT", "")
RF_PROJECT = "fingerprint-pattern-detection-vmh4p"
RF_LABELLER_VER_NUM = 4
RF_LABELLER_PROJECT = os.environ.get("RF_LABELLER_PROJECT", None)
CLASSES = ["concentric_whorl", "imploding_whorl", "loop","standard_arch"]
print(RF_KEY, RF_WORKSPACE, RF_PROJECT, RF_LABELLER_PROJECT, RF_LABELLER_VER_NUM)

efcTvzGpArL7OorvmZT8 fingerprint-generation fingerprint-pattern-detection-vmh4p fingerprint-pattern-detection 4


## Saturating Image to further enhance the fingerprint ridges

In [140]:
def saturate_image(image_file_path):
    image = cv2.imread(f"{image_file_path}", cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(image)
    os.remove(image_file_path)

    cv2.imwrite(f"{image_file_path}", enhanced)

In [ ]:
def filter_images(class_name: str, dir: str):
    labels = {   
        "concentric_whorl": "0",
        "imploding_whorl": "1",
        "loop": "2",
        "standard_arch": "3"
    }

    label = labels.get(class_name.strip().lower(), None)
    label_dir = os.path.abspath(os.path.join(dir, "labels"))
    image_dir = os.path.abspath(os.path.join(dir, "images"))

    for label_file_name in os.listdir(label_dir):
        label_file_path = os.path.join(label_dir, label_file_name)
        try:
            with open(label_file_path, "r") as label_file:
                lines = label_file.read()
                image_file_name = re.sub(r"\.[^.]+$", "", label_file_name)+".jpg"
                image_file_path = os.path.join(image_dir, image_file_name)
                if lines[0] != label:
                    os.remove(label_file_path)
                    os.remove(image_file_path)
                    continue
                else:
                    if lines[0] != 1:
                        lines = lines.replace(lines[0], "0")
                    
                    saturate_image(image_file_path=image_file_path)
                    
            with open(label_file_path, "w") as label_file:
                 label_file.write(lines)

        except IndexError: # catches any images labelled as "null" on Roboflow
                os.remove(label_file_path)
                image_file_name = re.sub(r"\.[^.]+$", "", label_file_name)+".jpg"
                image_file_path = os.path.join(image_dir, image_file_name)
                os.remove(image_file_path)
        


In [142]:
def modify_yaml(class_name, dir):
    yaml_dir = os.path.abspath(os.path.join(dir, "data.yaml"))

    with open(yaml_dir, "r") as f:
        data = yaml.safe_load(f)
    data['names'] = [class_name]
    data['nc'] = 1
    with open(yaml_dir, 'w') as f:
        yaml.dump(data, f, sort_keys=False)    

In [143]:
def filter_for_class(dir:str, class_name:str):
    filter_images(class_name, os.path.abspath(os.path.join(dir, "train/")))
    filter_images(class_name, os.path.abspath(os.path.join(dir, "valid/")))
    filter_images(class_name, os.path.abspath(os.path.join(dir, "test/")))
    modify_yaml(class_name, dir)
    new_dir = os.path.abspath(os.path.join(dir, f"../v{RF_LABELLER_VER_NUM}/{class_name} dataset v{RF_LABELLER_VER_NUM}"))
    os.rename(dir, new_dir)

In [144]:
def filter_class_dataset(class_name: str):
    rf = Roboflow(api_key=RF_KEY)
    project = rf.workspace(RF_WORKSPACE).project(RF_PROJECT)
    dataset = project.version(RF_LABELLER_VER_NUM).download("yolov8", f"../../datasets/Fingerprint pattern/Fingerprint dataset v{RF_LABELLER_VER_NUM}")

    curr_dir = os.getcwd()
    parent_dir = os.path.abspath(os.path.join(curr_dir, f"../.."))
    dataset_dir = os.path.abspath(os.path.join(parent_dir, f"datasets/Fingerprint pattern/Fingerprint dataset v{RF_LABELLER_VER_NUM}" ))
    filter_for_class(dir = dataset_dir, class_name=class_name)

In [145]:
curr_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(curr_dir, f"../.."))
new_dir = os.path.abspath(os.path.join(parent_dir, f'datasets/Fingerprint pattern/v{RF_LABELLER_VER_NUM}'))
if os.path.exists(new_dir):
    shutil.rmtree(new_dir)
os.makedirs(new_dir)

for c in CLASSES:
    try:
        filter_class_dataset(c)
    except:
        pass


loading Roboflow workspace...
loading Roboflow project...
Generating version still in progress. Progress: 60.75%
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v4 in yolov8:: 100%|██████████| 8378/8378 [00:00<00:00, 9889.40it/s] 


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v4 in yolov8:: 100%|██████████| 8378/8378 [00:00<00:00, 10969.16it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v4 in yolov8:: 100%|██████████| 8378/8378 [00:00<00:00, 11257.50it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../../datasets/Fingerprint pattern/Fingerprint dataset v4 in yolov8:: 100%|██████████| 8378/8378 [00:00<00:00, 10724.75it/s]
